In [ ]:
import contractions

from nltk.stem import SnowballStemmer

import re

import pandas pd

Function to process text and create new features

In [ ]:
# Clean Text

def cleantext(text):

    text = text.lower().strip()
    text = re.sub(r'@[A-za-z0-9]+','',text) # remove @mentions
    text = re.sub(r'RT[\s]+','',text) #remove retweet
    text = re.sub(r'https?://\S+','',text) #remove hyperlink
    text = re.sub(r'#','',text) #remove #

    text = text.replace('\n',' ')

    text = contractions.fix(text)

    stemmer = SnowballStemmer(language='english')
    text = stemmer.stem(text)
    text = re.sub("[.,!?:;-=*'“()_]", "",text) # remove puntucation
    text = re.sub("[0123456789]", "", text) # remove number

    text = text.replace('  ','')

    return text.strip()
 
                                 
#Convert emoji symbol to test and extract to new colum 

def demoji(text):
    import emoji
    process_text=emoji.demojize(text)
    return process_text

def find_emoji(text):
    emoji_text=re.findall(':\w+:',text)
    return emoji_text
def create_emoji_col(df,text_col):
    df['emoji']=df[text_col].apply(lambda x:find_emoji(x) )
    return df

                  
                  
#modify hashtag col to only have hash tag text

def get_hashtag_text(text):
    text= re.findall("\: \'\w+\'",text)
    hash_tag_list=[]
    for i in text:
        hashtag=re.findall("\w+",i)
        hashtag=hashtag[0] 
        hash_tag_list.append(hashtag)
    
    return hash_tag_list

def modify_hashtags_col(df,hashtag_col):
    df['Hashtags']=df[hashtag_col].apply(lambda x: get_hashtag_text(x))
    return df

In [ ]:
df=pd.read_csv('fake_meat.csv',index_col=0)

#clean Text
df['Clean_text']=df['Tweet Text'].apply(lambda text : cleantext(text))

#Convert emoji symbol to test and extract to new colum 
df=create_emoji_col(df,'Tweet Text')


#modify hashtag col to only have hash tag text
df=modify_hashtags_col(df,'Hashtags')

In [ ]:
#saving to new csv
df.to_csv('fake_meat_clean.csv')